# Glacier Gallery

## Import packages

In [ ]:
import panel as pn
import os, numpy as np, pandas as pd, cartopy.crs as ccrs, bokeh
import holoviews as hv, geoviews as gv, datashader as ds
from bokeh.models import ColumnDataSource, OpenURL, TapTool, HoverTool, Plot, Range1d, Text, TapTool, CustomJS, Legend, LegendItem
from bokeh.plotting import figure, show, output_notebook
from bokeh.tile_providers import get_provider, Vendors
from bokeh.io import output_file, show, output_notebook, reset_output
from bokeh.models.markers import Circle
hv.extension('bokeh', width=100)
pn.extension()

### Function to prepare coordinates

In [ ]:
import math
from ast import literal_eval

def merc(Coords):
    lat = Coords[0]
    lon = Coords[1]

    r_major = 6378137.000
    x = r_major * math.radians(lon)
    scale = x/lon
    y = 180.0/math.pi * math.log(math.tan(math.pi/4.0 + 
        lat * (math.pi/180.0)/2.0)) * scale    
    return (x, y)

## Load data

In [ ]:
df_all_all = pd.read_hdf('./data/rgi62_era5_itmix_df.h5', 'df')
df_all_all['rgi_id']= list(df_all_all.axes[0])
df_all = df_all_all[['rgi_id', 'CenLon', 'CenLat', 'Area', 'Zmed','Lmax', 'Status','Connect', 'Form','TermType', 'Surging','Linkages', 'GlacierType','TerminusType', 'GlacierStatus', 'IsTidewater', 'IsNominal' ]].copy()
df_all.tail()

## Select glaciers, add photos and build lists with information about them

In [ ]:
# Important: fill the list with increasing rgi-ids!
# order of entries in tuples: 
                        # RGI-ID, Name, glacier type(Pure ice glacier, Debris covered glacier, Calving glacier, Ice cap),
                        # location from where picture is loaded into app,
                        # Photo courtesy text, license abbrevation, license link, 
                        # width image in hover, height image in hover, source link
            
Glacier_No = [('RGI60-01.01104', 'Lemon Creek Glacier', 'Pure ice glacier',
               'https://raw.githubusercontent.com/OGGM/glacier-gallery/master/glacier_photos/lemoncreek.jpg', 
               "Photo courtesy L. Bernier/National Snow and Ice Data Center",
               '', '',
               '390', '260',
              'http://nsidc.org/data/glacier_photo/search/image_info/lemoncreek20070917 '),
              
              ('RGI60-01.10689', 'Columbia Glacier', 'Calving glacier',
               'https://upload.wikimedia.org/wikipedia/commons/7/7b/Columbia_Glacier_%28Alaska%29_by_Sentinel-2.jpg', 
               'Photo contains modified Copernicus Sentinel-2 data, (European Space Agency - ESA)  2018', 
               '(CC BY-SA 3.0 IGO)', 'https://creativecommons.org/licenses/by-sa/3.0/igo/deed.en',
               '300', '318',
              'https://commons.wikimedia.org/wiki/File:Columbia_Glacier_(Alaska)_by_Sentinel-2.jpg '),
             
              ('RGI60-03.04539', 'White Glacier', 'Pure ice glacier',
               'https://raw.githubusercontent.com/OGGM/glacier-gallery/master/glacier_photos/white.jpg',
               'Photo courtesy J. Alean/National Snow and Ice Data Center',
               '', '',
               '300', '200', # this is set to very small outlines, otherwise it would be outside the margin of the browser window
              'http://nsidc.org/data/glacier_photo/search/image_info/white20080702'),
              
              ('RGI60-05.11186', 'Mittivakkat Glacier', 'Ice cap',
               'https://raw.githubusercontent.com/OGGM/glacier-gallery/master/glacier_photos/mittivakkat.jpg', 
               'Photo courtesy N. Knudsen/National Snow and Ice Data Center',
               '', '',
               '400', '300',
              'http://nsidc.org/data/glacier_photo/search/image_info/mittivakkat20130827?order=true '),
              
              ('RGI60-07.00570', 'Nordenskioldbreen', 'Calving glacier',
               'https://raw.githubusercontent.com/OGGM/glacier-gallery/master/glacier_photos/nordenskioldbreen.JPG', 
               'Photo courtesy H. Crités',
               '', '',
               '300', '225', # this is set to very small outlines, otherwise it would be outside the margin of the browser window
              ''), # https://www.researchgate.net/publication/325218515_Drumlins_in_the_Nordenskioldbreen_forefield_Svalbard           
              
              ('RGI60-08.01097', 'Briksdalsbreen', 'Pure ice glacier',
               'https://raw.githubusercontent.com/OGGM/glacier-gallery/master/glacier_photos/briksdalsbreen.jpg',
               'Photo courtesy J. Perez Crespo/National Snow and Ice Data Center',
               '(CC BY-NC)', '',
               '259', '387',
              'http://nsidc.org/data/glacier_photo/search/image_info/briksdalsbreen20090730?order=true'),
              
              ('RGI60-11.00897', 'Hintereisferner', 'Pure ice glacier',
               'https://raw.githubusercontent.com/OGGM/glacier-gallery/master/glacier_photos/hintereisferner.bmp', 
               'Photo courtesy A. Lambrecht/National Snow and Ice Data Center',
               '', '',
               '400', '265',
              'http://nsidc.org/data/glacier_photo/search/image_info/hintereisferner20060912?order=true'), 
              
              ('RGI60-11.01450', 'Großer Aletschgletscher', 'Pure ice glacier',
               'https://upload.wikimedia.org/wikipedia/commons/a/a1/Grosser_Aletschgletscher_3196.JPG', 
               'Photo courtesy Dirk Beyer',
               '(CC BY-SA 3.0)', 'https://creativecommons.org/licenses/by-sa/3.0/',
               '408', '306',
              'https://commons.wikimedia.org/wiki/File:Grosser_Aletschgletscher_3196.JPG'), 
              
              ('RGI60-12.00411', 'Tbilisa Glacier', 'Pure ice glacier',
               'https://raw.githubusercontent.com/OGGM/glacier-gallery/master/glacier_photos/tbilisa.jpg', 
               'Photo courtesy L. Tielidze/National Snow and Ice Data Center',
               '', '',
               '360', '272',
              'http://nsidc.org/data/glacier_photo/search/image_info/tbilisa20109999'),
              
              ('RGI60-15.03733', 'Khumbu Glacier', 'Debris covered glacier',
               'https://raw.githubusercontent.com/OGGM/glacier-gallery/master/glacier_photos/khumbu.JPG', 
               'Photo courtesy Z. Schirmeister',
               '', '',
               '400', '300',
              ''),

              ('RGI60-16.01638', 'Lewis', 'Pure ice glacier',
               'https://raw.githubusercontent.com/OGGM/glacier-gallery/master/glacier_photos/lewis.jpg', 
               'Photo courtesy R. Prinz/National Snow and Ice Data Center',
               '', '',
               '400', '300',
              'http://nsidc.org/data/glacier_photo/search/image_info/lewis20120223'),
              
              ('RGI60-16.02444', 'Artesonraju', 'Pure ice glacier',
               'https://upload.wikimedia.org/wikipedia/commons/0/06/Artesonraju_Peru.jpg', 
               'Photo courtesy Jonas Jancarik',
               '(CC BY 3.0)', 'https://creativecommons.org/licenses/by/3.0/deed.en',
               '400', '276',
              'https://commons.wikimedia.org/wiki/File:Artesonraju_Peru.jpg'), 
              
              ('RGI60-17.00312', 'Perito Moreno', 'Calving glacier',
               'https://raw.githubusercontent.com/OGGM/glacier-gallery/master/glacier_photos/peritomoreno.jpg', 
               'Imagery from International Space Station, courtesy Earth Science and Remote Sensing Unit, NASA Johnson Space Center',
               '', '',
               '400', '272',
              'https://eol.jsc.nasa.gov/SearchPhotos/photo.pl?mission=ISS016&roll=E&frame=24897'),
              
              ('RGI60-18.02342', 'Tasman Glacier', 'Debris covered glacier',
               'https://upload.wikimedia.org/wikipedia/commons/8/87/Lower_Tasman_Glacier_towards_Minarets.jpg', 
               '', 
               '(released into public domain by holder of the work)', '',
               '400', '266',
              'https://commons.wikimedia.org/wiki/File:Lower_Tasman_Glacier_towards_Minarets.jpg '),
              
              ('RGI60-19.00595', 'Bahia Del Diablo Glacier', 'Pure ice glacier',
               'https://raw.githubusercontent.com/OGGM/glacier-gallery/master/glacier_photos/bahiadeldiablo.bmp', 
               'Photo courtesy P. Skvarca/National Snow and Ice Data Center', 
               '', '',
               '400', '260',
              'http://nsidc.org/data/glacier_photo/search/image_info/bahiadeldiablo20040224')
             ]
# More glacier suggestions:
              #('RGI60-09.00114', 'Severny Island Ice Cap', np.NaN, np.NaN, '', ''),
              #('RGI60-09.00471', 'Vavilov Ice Cap', np.NaN, np.NaN,  '', ''),
              #('RGI60-13.04946', 'Samoilowich', np.NaN, np.NaN, '', ''), 
              #('RGI60-19.00386', 'Ross Island', '', '', ''),

# Create lists from glacier list above
glacier_list = []
glacier_names = []
gltype = []
glacier_pics =[]
glpic_cite = []
glpic_lic = []
glpic_wi = []
glpic_he = []
glpic_src = []
for gl_no in Glacier_No:
    glacier_list.append(gl_no[0])
    glacier_names.append(gl_no[1])
    gltype.append(gl_no[2])
    glacier_pics.append(gl_no[3])
    glpic_cite.append(gl_no[4])
    glpic_lic.append (gl_no[5])
    glpic_wi.append(gl_no[7])
    glpic_he.append(gl_no[8])
    glpic_src.append(gl_no[9])

# create a new dataframe with the selected glaciers from the loaded data about glaciers 
df = df_all.loc[df_all['rgi_id'].isin(glacier_list)].copy()
df['name'] = glacier_names
df['Coords'] = list(zip(round(df.CenLat,2), round(df.CenLon,2)))
df['type'] = gltype
df['img'] = glacier_pics
df['glpic_lic'] = glpic_lic
df['pic_cite'] = glpic_cite
df['pic_width'] = glpic_wi
df['pic_height'] = glpic_he
df['pic_src'] = glpic_src
df.tail()

## Add texts about the types to the dataframe

In [ ]:
# add glacier type description, #### Einzug wichtig!!!
Gl_title = 'Glaciers'
Gl_def = """
    Glaciers are defined as a perennial mass of 
    land ice that forms through the compression 
    of snow. The ice mass has a past or 
    present flow, which is caused by internal 
    deformation or sliding at the base. Glaciers 
    accumulate snow in high altitudes and loose 
    mass by ablation at low altitudes and/or 
    discharge into the sea or lakes.
    
    Further glaciers are separated into 
    unconstrained and constrained glaciers. 
    Unconstrained glaciers include ice sheets, 
    ice caps and ice streams. Constrained 
    glaciers include ice fields, valley glaciers, 
    transection glaciers (interconnected valley 
    glaciers), piedmont glaciers, cirque and 
    niche glaciers.
    """
Gl_text_source = """
    see IPCC, 2013: Climate Change 2013: The Physical 
    Science Basis. Contribution of Working Group I to 
    the Fifth Assessment Report of the Intergovern-
    mental Panel on Climate Change [Stocker, T.F., 
    D. Qin, G.-K. Plattner, M. Tignor, S.K. Allen, J. 
    Boschung, A. Nauels, Y. Xia, V. Bex and P.M. Midgley 
    (eds.)]. Cambridge University Press, Cambridge, 
    United Kingdom and New York, NY, USA, 1535 pp.,
    http://www.antarcticglaciers.org/glacier-processes
    /glacier-types/types-of-glaciers/
    """

pure_ice_text = """
    Most glaciers are pure ice glaciers. That 
    means that they are not covered by debris 
    nor riddled with rocks. They take shape all
    variants that are described in the glacier
    definition. You can find this kind of
    glacier on all continents.
    
    
    Please note that we put all glaciers in this
    category that do not fit to the other glacier 
    types listet, but exhibit the pure ice 
    characteristic. 
    """
pure_ice_source = """
    see IPCC, 2013: Climate Change 2013: The Physical 
    Science Basis. Contribution of Working Group I to 
    the Fifth Assessment Report of the Intergovernmental 
    Panel on Climate Change [Stocker, T.F., D. Qin, G.-K. 
    Plattner, M. Tignor, S.K. Allen, J. Boschung, 
    A. Nauels, Y. Xia, V. Bex and P.M. Midgley (eds.)]. 
    Cambridge University Press, Cambridge, United 
    Kingdom and New York, NY, USA, 1535 pp. and 
    http://www.antarcticglaciers.org/glacier-processes/
    glacier-types/types-of-glaciers/
"""

calving_glacier_text = '''
    A glacier calves when a huge mass of ice 
    breaks off at the front of the glacier. 
    Most of these glaciers are 
    water-terminating, that means they end
    in lakes, fjords or the sea. Either they 
    float on the water surface or they are 
    grounded. 
    Calving glaciers exhibit many crevasses 
    due to their relatively fast flow and
    an extensional (stretching) flow at 
    their lower end. Another feature is their
    near-vertical ice cliff where calving
    happens. Depending on the glacier 
    thickness the terminus is between ten to
    eighty meters thick. How often and how
    much a glacier calves is described by the
    calving rate, which depends on the 
    variations in the longitudinal strain 
    rates and the ice velocities.
    '''
calving_glacier_source = """
    see Bishop, M. P., Björnsson, H., Haeberli, W., 
    Oerlemans, J., Shroder, J. F., & Tranter, M. (2011).
    Encyclopedia of snow, ice and glaciers. Springer 
    Science & Business Media.
    """

debris_glacier_text = '''
    A layer of debris covers the lower part of 
    the glacier’s surface. This type of 
    glacier can be found in all major mountain 
    regions of the world, e.g. in the Himalayan 
    it is the predominant form of glaciation.
    The layer of debris forms by rockfall from 
    nearby mountain faces with high 
    denudation rates, rock avalanching and 
    mixed snow/ice/rock avalanching. The 
    resulting distribution and type of the debris 
    layer influences the glacier’s dynamics, 
    rheology and mass balance, and especially 
    its ablation. Depending on the thickness of 
    the layer, melting increases or decreases: 
    Dispersed debris cover and layers thinner 
    than ~15-80 mm enhance melting, because 
    the rocks heat up under solar radiation. At 
    the contrary thicker layers effect an 
    insulating effect and protect the ice from 
    insolation.
    '''
debris_glacier_source = '''    
    see Bishop, M. P., Björnsson, H., Haeberli, W., 
    Oerlemans, J., Shroder, J. F., & Tranter, M. (2011).
    Encyclopedia of snow, ice and glaciers. Springer 
    Science & Business Media.
    ''' 
ice_cap_text = '''
    Ice caps are defined as huge ice masses 
    that cover land on a large scale and 
    do not extent 50000 km² (approximately 
    the area of Costa Rica). This definition
    excludes Greenland and Antartica ice 
    sheets, which are much larger.
    Ice caps can be found in polar or 
    subtemperate regions, where 
    accumulation of snow commonly exceeds 
    ablation. The formation process is 
    described in the following: glaciers 
    grow in the higher parts of the mountains, 
    nearby glaciers merge and build ice bands 
    around the mountains. These ice bands 
    fuse with the ones from other mountains. 
    As a result they build the ice caps 
    that cover large parts of the land. 
    Ice caps submerge the topography beneath 
    while only some peaks stick out of the 
    ice, which are called “nunataks”. 
    '''
ice_cap_source = '''        
    see Bishop, M. P., Björnsson, H., Haeberli, W., 
    Oerlemans, J., Shroder, J. F., & Tranter, M. (2011).
    Encyclopedia of snow, ice and glaciers. Springer 
    Science & Business Media.
    '''

df['Def'] = Gl_def
df['Title'] = Gl_title
df['text_source'] = Gl_text_source

df.loc[df['type'] == 'Pure ice glacier', 'glaciertype_text'] = pure_ice_text 
df.loc[df['type'] == 'Calving glacier', 'glaciertype_text'] = calving_glacier_text
df.loc[df['type'] == 'Debris covered glacier', 'glaciertype_text'] = debris_glacier_text
df.loc[df['type'] == 'Ice cap', 'glaciertype_text'] = ice_cap_text
df.loc[df['type'] == 'Pure ice glacier', 'glaciertype_source'] = pure_ice_source
df.loc[df['type'] == 'Calving glacier', 'glaciertype_source'] = calving_glacier_source
df.loc[df['type'] == 'Debris covered glacier', 'glaciertype_source'] = debris_glacier_source
df.loc[df['type'] == 'Ice cap', 'glaciertype_source'] = ice_cap_source

# define colors for dots of different glacier types
df.loc[df['type'] == 'Pure ice glacier', 'color'] = '#85a9c1'
df.loc[df['type'] == 'Calving glacier', 'color'] = '#FD90A7'
df.loc[df['type'] == 'Debris covered glacier', 'color'] = '#9FBC40'
df.loc[df['type'] == 'Ice cap', 'color'] = '#FFFED9'

# 3 empty columns for changing text box
df['empty1'] = ''
df['empty2'] = ''
df['empty3'] = ''

## Build special datastructure about all information

In [ ]:
# prepare coordinates  
df.loc[:, 'coords_x'] = df['Coords'].apply(lambda x: merc(x)[0])
df.loc[:, 'coords_y'] = df['Coords'].apply(lambda x: merc(x)[1])

# build sources
source = ColumnDataSource(df)

## Design for map with hover and webpage

In [ ]:
# Hover
TOOLTIPS = """
    <div>
        <div>
            <img
                src="@img" 
                height="@pic_height" 
                alt="image loading..." 
                width="@pic_width"
                style=" margin: 2px 2px 0px 2px;"
                border="2">
            </img>
        </div>
        <div>
            <span style="font-size: 17px; font-weight: bold;">@name                </span> <br>
            <span style="font-size: 15px;                   ">@type                </span> <br>
            <span style="font-size: 15px;                   ">Length:              </span>
            <span style="font-size: 15px;                   ">@Lmax                </span>
            <span style="font-size: 15px;                   ">m                    </span> <br>
            <span style="font-size: 15px;                   ">@TerminusType        </span> <br>
            <span style="font-size: 15px;                   ">Location             </span>
            <span style="font-size: 13px; color: #696;      ">(@CenLat, @CenLon)   </span> <br>
            <span style="font-size: 9px;                    ">@pic_cite @glpic_lic </span>
        </div>
    </div>
"""
hover = HoverTool()
hover.tooltips = TOOLTIPS

############################################################################
# Create figure for the map
p = figure(x_axis_type="mercator", 
           y_axis_type="mercator",
           plot_width = 1000,
           plot_height = 650,
           background='black',
           tools=['tap', 'wheel_zoom', 'reset', 'pan'])

############################################################################
# Design figure
p.xaxis.visible = False
p.yaxis.visible = False
p.xgrid.visible = False
p.ygrid.visible = False
p.xgrid.grid_line_color = None

############################################################################
# add map
tile_provider = get_provider(Vendors.CARTODBPOSITRON)
p.add_tile(tile_provider)
# add glacier dots
p.circle(x='coords_x',
         y='coords_y', 
         source=source,
         size=10,
         color='color',
         line_color="black", 
         fill_alpha=0.9,
         legend='type'
        )
# add markers around the dots when mouse is directly over glacier dots (belongs to hover)
cr = p.circle(x='coords_x', y='coords_y', source=source, size=20,
                fill_color="white", hover_fill_color="blue",
                fill_alpha=0.0, hover_alpha=0.3,
                line_color=None, hover_line_color="white")
# add hover
hover = HoverTool(tooltips=TOOLTIPS, renderers=[cr])
p.add_tools(hover)

############################################################################
# build text boxes 

# glacier type text box
def construct_changing_text_box(source): 
    # Plot and axes                                                             
    xdr = Range1d(0, 1220)                                                       
    ydr = Range1d(0, 1120)                                                       

    plot = Plot(                                        
        x_range=xdr, y_range=ydr,                                   
        plot_height=650, min_border=0, 
        plot_width=350
        )                                                                           
    # Add the writing 
    title = Text(x=10, y=1055, text='empty1', text_color="#85a9c1", text_alpha=0.2, text_font='helvetica', text_font_size='14pt')
    text  = Text(x=10, y=310,  text='empty2', text_color="#326a86", text_alpha=0.2, text_font='helvetica', text_font_size='10pt')
    sourc = Text(x=10, y=5,     text='empty3', text_color="#85a9c1", text_alpha=0.2, text_font='helvetica', text_font_size='8pt')
    
    plot.add_glyph(source, text)
    plot.add_glyph(source, title)    
    plot.add_glyph(source, sourc)      
    return plot

# Glacier definition text box
def construct_text_box(source): 
    # Plot and axes                                                             
    xdr = Range1d(0, 1220)                                                       
    ydr = Range1d(0, 1120)                                                       

    plot = Plot(                                        
        x_range=xdr, y_range=ydr,                                   
        plot_height=650, min_border=0, 
        plot_width=350
        )                                                                           
    # Add the writing 
    title1 = Text(x=10, y=1055, text='Title', text_color="#85a9c1", text_alpha=0.2, text_font='helvetica', text_font_size='14pt')
    text1 = Text(x=10, y=300, text='Def', text_color="#326a86", text_alpha=0.2, text_font='helvetica', text_font_size='10pt')
    cite1 = Text(x=10, y=5, text='text_source', text_color="#85a9c1", text_alpha=0.2, text_font='helvetica', text_font_size='8pt')

    plot.add_glyph(source, title1)
    plot.add_glyph(source, text1)
    plot.add_glyph(source, cite1)
    return plot


JS_CODE ="""
    // Get index of current selected datapoint
    sel_point_index = cb_data.source.attributes.selected["1d"]["indices"][0];     

    /* replace all name_display values with the name_label
       value of currently selected point */
       
    for (i=0; i < cb_data.source.data.type.length; i++) {
        cb_data.source.data.empty1[i] = cb_data.source.data.type[sel_point_index];
        }       
    cb_data.source.change.emit();
    
    for (i=0; i < cb_data.source.data.type.length; i++) {
        cb_data.source.data.empty2[i] = cb_data.source.data.glaciertype_text[sel_point_index];
        }      
     cb_data.source.change.emit();

    for (i=0; i < cb_data.source.data.type.length; i++) {
        cb_data.source.data.empty3[i] = cb_data.source.data.glaciertype_source[sel_point_index];
        }      
     cb_data.source.change.emit();
    """

newtaptool = p.select(type=TapTool)
newtaptool.callback = CustomJS(code=JS_CODE)
changing_text_box = construct_changing_text_box(source)
text_box = construct_text_box(source)

# make 2 tabs (one for each box)
from bokeh.models.widgets import Panel, Tabs
tab2 = Panel(child=text_box, title="Glacier definition")
tab1 = Panel(child=changing_text_box, title="Glacier type")
tabs = Tabs(tabs=[ tab1, tab2 ])

## Design application

In [ ]:
# Panel
title       = '<div style="font-size:38px; color: #326a86; font-weight: bold" >Glacier Gallery</div>'
instruction = '<div style="font-size:15px; color: #326a86" >Explore glaciers of the world by moving the mouse over the colored dots on the map. When you <b>click</b> on them, information about <br>the glacier type will appear in the box on the right.</div>'
oggm_logo   = '<a href="http://edu.oggm.org"><img src="https://raw.githubusercontent.com/zschirmeister/glacier-gallery/master/oggm_loupe.png" width=220></a>'
pn_logo     = '<a href="https://panel.pyviz.org"><img src="http://panel.pyviz.org/_static/logo_stacked.png" width=60></a>'
fk_logo     = '<a href="https://www.uibk.ac.at/foerderkreis1669/"><img src="https://raw.githubusercontent.com/OGGM/world-glacier-explorer/master/img/logo_1669.png" width=180 height=79></a>'

links_to_license = """<div> 
                            Links to licenses of the images: <br>
                            <a href="https://creativecommons.org/licenses/by-sa/3.0/igo/deed.en">CC BY-SA 3.0 IGO </a> <br>
                            <a href="https://creativecommons.org/licenses/by-nc/4.0/"> CC BY-NC</a> <br>
                            <a href="https://creativecommons.org/licenses/by-sa/3.0/">CC BY-SA 3.0 </a> <br>
                            <a href="https://creativecommons.org/licenses/by/3.0/deed.en">CC BY 3.0 (Creative Commons Attribution 3.0 Unported)</a>
                      </div>"""
links_pic_src = """<div>
                        Links to sources of the images: <br>
                        <a href="http://nsidc.org/data/glacier_photo/search/image_info/lemoncreek20070917" >Lemon Creek Glacier</a> <br>
                        <a href="https://commons.wikimedia.org/wiki/File:Columbia_Glacier_(Alaska)_by_Sentinel-2.jpg" >Columbia Glacier</a> <br>
                        <a href="http://nsidc.org/data/glacier_photo/search/image_info/white2012059902?order=true" >White Glacier</a> <br>
                        <a href="http://nsidc.org/data/glacier_photo/search/image_info/mittivakkat20130827?order=true" > Mittivakkat Glacier</a> <br>
                        <a href="http://nsidc.org/data/glacier_photo/search/image_info/briksdalsbreen20090730?order=true" >Briksdalsbreen</a> <br>
                        <a href="http://nsidc.org/data/glacier_photo/search/image_info/hintereisferner20060912?order=true" >Hintereisferner</a> <br>
                        <a href="https://commons.wikimedia.org/wiki/File:Grosser_Aletschgletscher_3196.JPG" >Großer Aletschgletscher</a> <br>
                        <a href="http://nsidc.org/data/glacier_photo/search/image_info/tbilisa20109999" >Tbilisa Glacier</a> <br>
                        <a href="http://nsidc.org/data/glacier_photo/search/image_info/lewis20120223" >Lewis Glacier</a> <br>
                        <a href="https://commons.wikimedia.org/wiki/File:Artesonraju_Peru.jpg" >Artesonraju</a> <br>
                        <a href="https://eol.jsc.nasa.gov/SearchPhotos/photo.pl?mission=ISS016&roll=E&frame=24897" >Perito Moreno</a> <br>
                        <a href="https://commons.wikimedia.org/wiki/File:Lower_Tasman_Glacier_towards_Minarets.jpg" >Tasman Glacier</a> <br>
                        <a href="http://nsidc.org/data/glacier_photo/search/image_info/bahiadeldiablo20040224" >Bahia Del Diablo Glacier</a>
                   </div>"""

header = pn.Row(pn.Pane(oggm_logo),  pn.layout.Spacer(width=30), 
                pn.Column(pn.Row(pn.Spacer(width=130), pn.Pane(title, width=400)), pn.Pane(instruction, width=1200)),
                pn.layout.HSpacer()
                )
logos = pn.Column(pn.Spacer(height=30), pn.Pane(pn_logo, width=100), pn.Spacer(height=40), pn.Pane(fk_logo))
links = pn.Column(links_to_license, links_pic_src)

# Put plot and text together and show it
app = pn.Column(header, pn.Row(logos, pn.Column(pn.Spacer(height=22), p), pn.Spacer(width=5), tabs), pn.Spacer(height=60), links, width_policy='max', height_policy='max')
app.show()

In [ ]:
# put transform app to one model:
app_to_save = app.get_root()

from bokeh.resources import CDN
from bokeh.embed import file_html
# save model in html
bokeh.io.save(app_to_save, 'gallery', CDN)